In [ ]:
!pip install gradio
!pip install -q kokoro>=0.9.2 soundfile
!apt-get -qq -y install espeak-ng > /dev/null 2>&1

In [ ]:
# Import library
from kokoro import KPipeline
import soundfile as sf
import gradio as gr
import torch
import numpy as np

# Inisialisasi pipeline
pipeline = KPipeline(lang_code='a')  # Default, akan otomatis sesuai voice code

# Voice dictionary
voices = {
    'American English': {
        'Female': {'Heart': 'af_heart', 'Bella': 'af_bella'},
        'Male': {'Adam': 'am_adam', 'Michael': 'am_michael'}
    },
    'British English': {
        'Female': {'Emma': 'bf_emma', 'Isabella': 'bf_isabella'},
        'Male': {'George': 'bm_george', 'Lewis': 'bm_lewis'}
    },
    'Japanese': {
        'Female': {'Alpha': 'jf_alpha', 'nezumi': 'jf_nezumi'},
        'Male': {'Kumo': 'jm_kumo'}
    },
    'Chinese': {
        'Female': {'xiaobei': 'zf_xiaobei', 'xiaoni': 'zf_xiaoni'},
        'Male': {'yunjian': 'zm_yunjian', 'yunxi': 'zm_yunxi'}
    },
    'French': {
        'Female': {'Siwis': 'ff_siwis'}
    },
    'Spanish': {
        'Female': {'Dora': 'ef_dora'},
        'Male': {'Alex': 'em_alex', 'Santa': 'em_santa'}
    },
    'Hindi': {
        'Female': {'Alpha': 'hf_alpha', 'Beta': 'hf_beta'},
        'Male': {'Omega': 'hm_omega', 'Psi': 'hm_psi'}
    },
    'Italian': {
        'Female': {'Sara': 'if_sara'},
        'Male': {'Nicola': 'im_nicola'}
    },
    'Brazilian Portuguese': {
        'Female': {'Dora': 'pf_dora'},
        'Male': {'Alex': 'pm_alex', 'Santa': 'pm_santa'}
    },
}

# Fungsi untuk menghasilkan TTS
def kokoro_tts(text, language, gender, actor_name):
    voice_code = voices[language][gender][actor_name]
    generator = pipeline(text, voice=voice_code)
    all_audio = []
    for _, _, audio in generator:
        all_audio.extend(audio)
    audio_np = np.array(all_audio, dtype=np.float32)
    return (24000, audio_np)

# Update gender list
def update_genders(language):
    genders = list(voices[language].keys())
    return gr.update(choices=genders, value=genders[0])

# Update actor list
def update_actors(language, gender):
    actors = list(voices[language][gender].keys())
    return gr.update(choices=actors, value=actors[0])

# Interface
with gr.Blocks(theme=gr.themes.Soft()) as interface:
    gr.Markdown("""
    # 🎙️ Kokoro TTS - Multilingual Voice Synthesizer
    <p style='font-size:16px;'>Ubah teks menjadi suara dengan berbagai karakter dari banyak bahasa! Pilih bahasa, gender, dan pengisi suara favoritmu.</p>
    """)

    with gr.Row(equal_height=True):
        with gr.Column(scale=1):
            language = gr.Dropdown(label="🌐 Bahasa", choices=list(voices.keys()), value="American English")
            gender = gr.Dropdown(label="👤 Jenis Suara", choices=list(voices["American English"].keys()), value="Female")
            actor_name = gr.Dropdown(label="🎭 Voice Actor", choices=list(voices["American English"]["Female"].keys()), value="Bella")
        with gr.Column(scale=2):
            text_input = gr.Textbox(lines=5, placeholder="Ketik teks di sini...", label="📝 Masukkan Teks")

    with gr.Row():
        generate_btn = gr.Button("🔊 Generate Suara", variant="primary")
        audio_output = gr.Audio(label="🎧 Output Audio", type="numpy")

    # Tombol klik untuk menghasilkan suara
    generate_btn.click(fn=kokoro_tts, inputs=[text_input, language, gender, actor_name], outputs=audio_output)

    # Perubahan bahasa akan perbarui gender dan actor
    language.change(fn=update_genders, inputs=language, outputs=gender)
    language.change(fn=lambda lang: update_actors(lang, list(voices[lang].keys())[0]), inputs=language, outputs=actor_name)

    # Perubahan gender akan memperbarui actor
    gender.change(fn=lambda gender, lang: update_actors(lang, gender), inputs=[gender, language], outputs=actor_name)

# Jalankan antarmuka
interface.launch()
